In [28]:
print("Hello World")

Hello World


In [29]:
import requests
import pandas as pd
from datetime import datetime

In [ ]:
GITHUB_TOKEN = "YOUR_GITHUB_TOKEN_HERE"

HEADERS = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json"
}

In [31]:
url = "https://api.github.com/search/repositories"
params = {
    "q": "language:python",
    "sort": "stars",
    "order": "desc",
    "per_page": 50
}
try:
    response = requests.get(url, headers=HEADERS, params=params, timeout=10)
    response.raise_for_status()
    data = response.json()
    if data:
        print("Data fetched successfully")

except requests.exceptions.Timeout:
    raise Exception("Request timed out")

except requests.exceptions.HTTPError as e:
    raise Exception(f"HTTP error occurred: {e}")

except requests.exceptions.RequestException as e:
    raise Exception(f"Request failed: {e}")

Data fetched successfully


In [32]:
items = data.get("items", [])
len(items)

50

In [33]:
df = pd.json_normalize(items)
df.head()

,id,node_id,name,full_name,private,html_url,description,fork,url,forks_url,...,license.name,license.spdx_id,license.url,license.node_id,permissions.admin,permissions.maintain,permissions.push,permissions.triage,permissions.pull,license
0,54346799,MDEwOlJlcG9zaXRvcnk1NDM0Njc5OQ==,public-apis,public-apis/public-apis,False,https://github.com/public-apis/public-apis,A collective list of free APIs,False,https://api.github.com/repos/public-apis/publi...,https://api.github.com/repos/public-apis/publi...,...,MIT License,MIT,https://api.github.com/licenses/mit,MDc6TGljZW5zZTEz,False,False,False,False,True,NaN
1,13491895,MDEwOlJlcG9zaXRvcnkxMzQ5MTg5NQ==,free-programming-books,EbookFoundation/free-programming-books,False,https://github.com/EbookFoundation/free-progra...,:books: Freely available programming books,False,https://api.github.com/repos/EbookFoundation/f...,https://api.github.com/repos/EbookFoundation/f...,...,Creative Commons Attribution 4.0 International,CC-BY-4.0,https://api.github.com/licenses/cc-by-4.0,MDc6TGljZW5zZTI1,False,False,False,False,True,NaN
2,83222441,MDEwOlJlcG9zaXRvcnk4MzIyMjQ0MQ==,system-design-primer,donnemartin/system-design-primer,False,https://github.com/donnemartin/system-design-p...,Learn how to design large-scale systems. Prep ...,False,https://api.github.com/repos/donnemartin/syste...,https://api.github.com/repos/donnemartin/syste...,...,Other,NOASSERTION,None,MDc6TGljZW5zZTA=,False,False,False,False,True,NaN
3,21289110,MDEwOlJlcG9zaXRvcnkyMTI4OTExMA==,awesome-python,vinta/awesome-python,False,https://github.com/vinta/awesome-python,An opinionated list of awesome Python framewor...,False,https://api.github.com/repos/vinta/awesome-python,https://api.github.com/repos/vinta/awesome-pyt...,...,Other,NOASSERTION,None,MDc6TGljZW5zZTA=,False,False,False,False,True,NaN
4,63476337,MDEwOlJlcG9zaXRvcnk2MzQ3NjMzNw==,Python,TheAlgorithms/Python,False,https://github.com/TheAlgorithms/Python,All Algorithms implemented in Python,False,https://api.github.com/repos/TheAlgorithms/Python,https://api.github.com/repos/TheAlgorithms/Pyt...,...,MIT License,MIT,https://api.github.com/licenses/mit,MDc6TGljZW5zZTEz,False,False,False,False,True,NaN


In [34]:
df.isna().sum()

id                       0
node_id                  0
name                     0
full_name                0
private                  0
                        ..
permissions.maintain     0
permissions.push         0
permissions.triage       0
permissions.pull         0
license                 50
Length: 108, dtype: int64

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Columns: 108 entries, id to license
dtypes: bool(19), float64(2), int64(10), object(77)
memory usage: 35.8+ KB


In [36]:
columns = [
    "name",
    "full_name",
    "owner.login",
    "stargazers_count",
    "forks_count",
    "open_issues_count",
    "language",
    "created_at",
    "updated_at",
    "license.name",
    "html_url"
]

df = df[columns]
df.head()


,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url
0,public-apis,public-apis/public-apis,public-apis,384175,40992,750,Python,2016-03-20T23:49:42Z,2025-12-14T20:44:55Z,MIT License,https://github.com/public-apis/public-apis
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378803,65626,204,Python,2013-10-11T06:50:37Z,2025-12-14T20:47:43Z,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329357,53629,516,Python,2017-02-26T16:15:28Z,2025-12-14T21:01:30Z,Other,https://github.com/donnemartin/system-design-p...
3,awesome-python,vinta/awesome-python,vinta,273749,26898,526,Python,2014-06-27T21:00:06Z,2025-12-14T20:41:45Z,Other,https://github.com/vinta/awesome-python
4,Python,TheAlgorithms/Python,TheAlgorithms,214791,49609,793,Python,2016-07-16T09:44:01Z,2025-12-14T21:02:58Z,MIT License,https://github.com/TheAlgorithms/Python


In [37]:
df_clean = df.dropna(subset=["language"])
df_clean.head()

,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url
0,public-apis,public-apis/public-apis,public-apis,384175,40992,750,Python,2016-03-20T23:49:42Z,2025-12-14T20:44:55Z,MIT License,https://github.com/public-apis/public-apis
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378803,65626,204,Python,2013-10-11T06:50:37Z,2025-12-14T20:47:43Z,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329357,53629,516,Python,2017-02-26T16:15:28Z,2025-12-14T21:01:30Z,Other,https://github.com/donnemartin/system-design-p...
3,awesome-python,vinta/awesome-python,vinta,273749,26898,526,Python,2014-06-27T21:00:06Z,2025-12-14T20:41:45Z,Other,https://github.com/vinta/awesome-python
4,Python,TheAlgorithms/Python,TheAlgorithms,214791,49609,793,Python,2016-07-16T09:44:01Z,2025-12-14T21:02:58Z,MIT License,https://github.com/TheAlgorithms/Python


In [38]:
df_clean["license.name"] = df_clean["license.name"].fillna("No License")
df_clean.head()

,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url
0,public-apis,public-apis/public-apis,public-apis,384175,40992,750,Python,2016-03-20T23:49:42Z,2025-12-14T20:44:55Z,MIT License,https://github.com/public-apis/public-apis
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378803,65626,204,Python,2013-10-11T06:50:37Z,2025-12-14T20:47:43Z,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329357,53629,516,Python,2017-02-26T16:15:28Z,2025-12-14T21:01:30Z,Other,https://github.com/donnemartin/system-design-p...
3,awesome-python,vinta/awesome-python,vinta,273749,26898,526,Python,2014-06-27T21:00:06Z,2025-12-14T20:41:45Z,Other,https://github.com/vinta/awesome-python
4,Python,TheAlgorithms/Python,TheAlgorithms,214791,49609,793,Python,2016-07-16T09:44:01Z,2025-12-14T21:02:58Z,MIT License,https://github.com/TheAlgorithms/Python


In [39]:
df_clean["created_at"] = pd.to_datetime(df_clean["created_at"])
df_clean["updated_at"] = pd.to_datetime(df_clean["updated_at"])
df_clean.head()

,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url
0,public-apis,public-apis/public-apis,public-apis,384175,40992,750,Python,2016-03-20 23:49:42+00:00,2025-12-14 20:44:55+00:00,MIT License,https://github.com/public-apis/public-apis
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378803,65626,204,Python,2013-10-11 06:50:37+00:00,2025-12-14 20:47:43+00:00,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329357,53629,516,Python,2017-02-26 16:15:28+00:00,2025-12-14 21:01:30+00:00,Other,https://github.com/donnemartin/system-design-p...
3,awesome-python,vinta/awesome-python,vinta,273749,26898,526,Python,2014-06-27 21:00:06+00:00,2025-12-14 20:41:45+00:00,Other,https://github.com/vinta/awesome-python
4,Python,TheAlgorithms/Python,TheAlgorithms,214791,49609,793,Python,2016-07-16 09:44:01+00:00,2025-12-14 21:02:58+00:00,MIT License,https://github.com/TheAlgorithms/Python


In [41]:
today = pd.Timestamp.utcnow()

df_clean["repo_age_days"] = (today - df_clean["created_at"]).dt.days

df_clean["stars_forks_ratio"] = (
    df_clean["stargazers_count"] /
    df_clean["forks_count"].replace(0, 1)
)

df_clean.head()


,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url,repo_age_days,stars_forks_ratio
0,public-apis,public-apis/public-apis,public-apis,384175,40992,750,Python,2016-03-20 23:49:42+00:00,2025-12-14 20:44:55+00:00,MIT License,https://github.com/public-apis/public-apis,3555,9.371951
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378803,65626,204,Python,2013-10-11 06:50:37+00:00,2025-12-14 20:47:43+00:00,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...,4447,5.772148
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329357,53629,516,Python,2017-02-26 16:15:28+00:00,2025-12-14 21:01:30+00:00,Other,https://github.com/donnemartin/system-design-p...,3213,6.141397
3,awesome-python,vinta/awesome-python,vinta,273749,26898,526,Python,2014-06-27 21:00:06+00:00,2025-12-14 20:41:45+00:00,Other,https://github.com/vinta/awesome-python,4188,10.177299
4,Python,TheAlgorithms/Python,TheAlgorithms,214791,49609,793,Python,2016-07-16 09:44:01+00:00,2025-12-14 21:02:58+00:00,MIT License,https://github.com/TheAlgorithms/Python,3438,4.329678


In [42]:
df_filtered = df_clean[
    (df_clean["stargazers_count"] > 10_000) &
    (df_clean["repo_age_days"] > 365) &
    (df_clean["owner.login"].str.len() > 2)
]

df_filtered.head()

,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url,repo_age_days,stars_forks_ratio
0,public-apis,public-apis/public-apis,public-apis,384175,40992,750,Python,2016-03-20 23:49:42+00:00,2025-12-14 20:44:55+00:00,MIT License,https://github.com/public-apis/public-apis,3555,9.371951
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378803,65626,204,Python,2013-10-11 06:50:37+00:00,2025-12-14 20:47:43+00:00,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...,4447,5.772148
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329357,53629,516,Python,2017-02-26 16:15:28+00:00,2025-12-14 21:01:30+00:00,Other,https://github.com/donnemartin/system-design-p...,3213,6.141397
3,awesome-python,vinta/awesome-python,vinta,273749,26898,526,Python,2014-06-27 21:00:06+00:00,2025-12-14 20:41:45+00:00,Other,https://github.com/vinta/awesome-python,4188,10.177299
4,Python,TheAlgorithms/Python,TheAlgorithms,214791,49609,793,Python,2016-07-16 09:44:01+00:00,2025-12-14 21:02:58+00:00,MIT License,https://github.com/TheAlgorithms/Python,3438,4.329678


In [44]:
print(df_filtered.shape)
print(df_filtered.isnull().sum())

(49, 13)
name                 0
full_name            0
owner.login          0
stargazers_count     0
forks_count          0
open_issues_count    0
language             0
created_at           0
updated_at           0
license.name         0
html_url             0
repo_age_days        0
stars_forks_ratio    0
dtype: int64


In [46]:
output_file = "cleaned_popular_python_repos.csv"
df_filtered.to_csv(output_file, index=False)
output_file


'cleaned_popular_python_repos.csv'